In [1]:
!nvidia-smi

Thu Apr 25 16:02:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:0A:00.0 Off |                    0 |
| N/A   31C    P0              72W / 700W |      0MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

!pip install huggingface_hub
!huggingface-cli login
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --cache-dir $TMP_DIR --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

In [3]:

import os, sys
script_dir = os.getcwd()
module_path = script_dir
for _ in range(1):
    module_path = os.path.abspath(os.path.join(module_path, '../'))
    if module_path not in sys.path:
        sys.path.insert(0,module_path)
        
sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
import time
import gc
from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


In [4]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2  

In [5]:
%reload_ext autoreload

In [6]:
import random

# List of prompts
prompts = [
    "What are the main causes of climate change?",
    "How do you bake a chocolate cake from scratch?",
    "What is the plot of Shakespeare's 'Macbeth'?",
    "Can you explain the theory of relativity in simple terms?",
    "What are the health benefits of daily exercise?",
    "How does a blockchain work?",
    "What are the top tourist attractions in Paris?",
    "What is the process of photosynthesis in plants?",
    "How can one improve their time management skills?",
    "What are the rules of chess?"
]

# Sampling and printing the selected prompts
def get_prompts(b: int = 2):
    random.shuffle(prompts)
    return prompts[:b]

In [11]:
from transformers import TextStreamer

def run_for_config(offload_per_layer: int, attn_bits: int, ffn_bits: int, routing_strategy: str, alpha: float, n: int = 5, b_size:int = 1):
    print (f"Running with configs: {offload_per_layer}, {attn_bits}, {ffn_bits}, {routing_strategy}, {alpha}, {n}, {b_size}")
    model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
    quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
    state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"
    
    config = AutoConfig.from_pretrained(quantized_model_name)
    
    device = torch.device("cuda:0")
    
    ##### Change this to 5 if you have only 12 GB of GPU VRAM #####
    # offload_per_layer = 5
    ###############################################################
    
    num_experts = config.num_local_experts
    
    offload_config = OffloadConfig(
        main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
        offload_size=config.num_hidden_layers * offload_per_layer,
        buffer_size=4,
        offload_per_layer=offload_per_layer,
    )
    
    attn_config = BaseQuantizeConfig(
        nbits=attn_bits,
        group_size=64,
        quant_zero=True,
        quant_scale=True,
    )
    attn_config["scale_quant_params"]["group_size"] = 256
    
    
    ffn_config = BaseQuantizeConfig(
        nbits=ffn_bits,
        group_size=16,
        quant_zero=True,
        quant_scale=True,
    )
    quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)

    if 'model' in locals():
        print("Deleting existing model")
        del model
    
    model = build_model(
        device=device,
        quant_config=quant_config,
        offload_config=offload_config,
        state_path=state_path,
        routing_strategy=routing_strategy,
        routing_threshold=alpha
    )

    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    past_key_values = None
    sequence = None
    
    seq_len = 0
    # while True:
    #user_input = "Where is Georgia Tech? What is the name of its mascot?"
    #user_entry = dict(role="user", content=user_input)
    batch_prompts = get_prompts(b_size)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True)
    input_ids = inputs['input_ids'].to(model.device)
    print (f"Batch: {batch_prompts} | Input_ids: {input_ids.shape}")
    
    if past_key_values is None:
      attention_mask = torch.ones(input_ids.shape).long().to(device=model.device)
    else:
      seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
      attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=model.device)

    results = []
    
    for idx in range(n):
        start_time = time.time()
        result = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # streamer=streamer,
            do_sample=False,
            # temperature=0.9,
            # top_p=0.9,
            min_new_tokens=100,
            max_new_tokens=100,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            # output_hidden_states=True,
            # decoder_router_logits=True, 
            output_router_logits=True,
            # output_logits = False
        )
        latency = time.time() - start_time
        total_experts_saved = 0
        for i in result['router_logits'][-32:]:
            total_experts_saved += i[1]
        total_experts_saved 
        
        Num_tokens = result['sequences'].shape[1] - input_ids.shape[1]
        print(f"Total Latency :{latency} sec, Throughput:{Num_tokens/latency}, Expert Loads saved:{total_experts_saved} ")

        results.append({"latency": latency, "throughput": Num_tokens/latency, "expert_loads_saved": total_experts_saved})

    return results

In [12]:
run_configs = [
    {"offload_per_layer":3, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'TOP-K', "alpha": 0.05, "n": 50},
    {"offload_per_layer":4, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'TOP-K', "alpha": 0.05, "n": 50},
    {"offload_per_layer":5, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'TOP-K', "alpha": 0.05, "n": 50},
    {"offload_per_layer":6, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'TOP-K', "alpha": 0.05, "n": 50},
    {"offload_per_layer":3, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.05, "n": 50},
    {"offload_per_layer":4, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.05, "n": 50},
    {"offload_per_layer":5, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.05, "n": 50},
    {"offload_per_layer":6, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.05, "n": 50},
    {"offload_per_layer":3, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.15, "n": 50},
    {"offload_per_layer":4, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.15, "n": 50},
    {"offload_per_layer":5, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.15, "n": 50},
    {"offload_per_layer":6, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.15, "n": 50},
    {"offload_per_layer":3, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.25, "n": 50},
    {"offload_per_layer":4, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.25, "n": 50},
    {"offload_per_layer":5, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.25, "n": 50},
    {"offload_per_layer":6, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'THRESHOLDING', "alpha": 0.25, "n": 50},
    {"offload_per_layer":3, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'BIASING', "alpha": 0.05, "n": 50},
    {"offload_per_layer":4, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'BIASING', "alpha": 0.05, "n": 50},
    {"offload_per_layer":5, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'BIASING', "alpha": 0.05, "n": 50},
    {"offload_per_layer":6, "attn_bits":4, "ffn_bits": 2, "routing_strategy":'BIASING', "alpha": 0.05, "n": 50},
]

print (len(run_configs))

20


In [13]:
profiling_results = []
from tqdm import tqdm
import pickle

for i, rc in tqdm(enumerate(run_configs)):
    result = run_for_config(**rc) # run_for_config(offload_per_layer=4, attn_bits=4, ffn_bits=2, routing_strategy='BIASING', alpha=0.05, n= 3)
    print (f"Config: {i} Result: {result}")
    profiling_results.append(result)

    with open("profiling_results.pkl", "wb") as fp:
        pickle.dump(profiling_results, fp)

0it [00:00, ?it/s]/scratch/615129/moe_offload/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Running with configs: 3, 4, 2, TOP-K, 0.05, 50, 1


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

Batch: ["What is the plot of Shakespeare's 'Macbeth'?"] | Input_ids: torch.Size([1, 15])
Total Latency :18.18439745903015 sec, Throughput:5.499219879311492, Expert Loads saved:0 
Total Latency :18.162853956222534 sec, Throughput:5.505742668031547, Expert Loads saved:0 
Total Latency :18.160635232925415 sec, Throughput:5.506415316282494, Expert Loads saved:0 
Total Latency :18.17613410949707 sec, Throughput:5.501719969580868, Expert Loads saved:0 
Total Latency :18.319108486175537 sec, Throughput:5.4587809267828025, Expert Loads saved:0 
Total Latency :18.17943525314331 sec, Throughput:5.500720930410065, Expert Loads saved:0 
Total Latency :18.221656322479248 sec, Throughput:5.487975309721676, Expert Loads saved:0 
Total Latency :18.224007606506348 sec, Throughput:5.487267244351782, Expert Loads saved:0 
Total Latency :18.28162169456482 sec, Throughput:5.469974254512131, Expert Loads saved:0 
Total Latency :18.214136838912964 sec, Throughput:5.4902409531896375, Expert Loads saved:0 
Tot

1it [15:33, 933.96s/it]

...

Total Latency :19.82347822189331 sec, Throughput:5.044523412120416, Expert Loads saved:53680 
Config: 18 Result: [{'latency': 20.062532663345337, 'throughput': 4.984415560987575, 'expert_loads_saved': 1087}, {'latency': 19.76108431816101, 'throughput': 5.060451055719503, 'expert_loads_saved': 2153}, {'latency': 19.920937299728394, 'throughput': 5.019844121559653, 'expert_loads_saved': 3247}, {'latency': 19.56750988960266, 'throughput': 5.110512301472541, 'expert_loads_saved': 4314}, {'latency': 19.965257167816162, 'throughput': 5.0087008226069445, 'expert_loads_saved': 5319}, {'latency': 19.542032957077026, 'throughput': 5.117174872217459, 'expert_loads_saved': 6343}, {'latency': 19.754781246185303, 'throughput': 5.062065671788203, 'expert_loads_saved': 7326}, {'latency': 19.01836371421814, 'throughput': 5.258075905091664, 'expert_loads_saved': 8420}, {'latency': 19.419079065322876, 'throughput': 5.149574790010122, 'expert_loads_saved': 9533}, {'latency': 19.77348804473877, 'throughput

Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

Batch: ['How can one improve their time management skills?'] | Input_ids: torch.Size([1, 10])
Total Latency :21.873337268829346 sec, Throughput:4.571776074723871, Expert Loads saved:1039 
Total Latency :21.73957848548889 sec, Throughput:4.599905194424525, Expert Loads saved:2070 
Total Latency :21.85190224647522 sec, Throughput:4.576260632692987, Expert Loads saved:3101 
Total Latency :21.84433627128601 sec, Throughput:4.577845660224899, Expert Loads saved:4132 
Total Latency :21.860982179641724 sec, Throughput:4.574359888236224, Expert Loads saved:5163 
Total Latency :22.102371215820312 sec, Throughput:4.524401432929628, Expert Loads saved:6194 
Total Latency :21.810094356536865 sec, Throughput:4.585032891892476, Expert Loads saved:7225 
Total Latency :21.9184091091156 sec, Throughput:4.562374919738642, Expert Loads saved:8256 
Total Latency :21.91471266746521 sec, Throughput:4.563144473642173, Expert Loads saved:9287 
Total Latency :21.94744563102722 sec, Throughput:4.556338887047040

20it [5:25:05, 975.29s/it]

Total Latency :21.674078702926636 sec, Throughput:4.6138062600325, Expert Loads saved:51558 
Config: 19 Result: [{'latency': 21.873337268829346, 'throughput': 4.571776074723871, 'expert_loads_saved': 1039}, {'latency': 21.73957848548889, 'throughput': 4.599905194424525, 'expert_loads_saved': 2070}, {'latency': 21.85190224647522, 'throughput': 4.576260632692987, 'expert_loads_saved': 3101}, {'latency': 21.84433627128601, 'throughput': 4.577845660224899, 'expert_loads_saved': 4132}, {'latency': 21.860982179641724, 'throughput': 4.574359888236224, 'expert_loads_saved': 5163}, {'latency': 22.102371215820312, 'throughput': 4.524401432929628, 'expert_loads_saved': 6194}, {'latency': 21.810094356536865, 'throughput': 4.585032891892476, 'expert_loads_saved': 7225}, {'latency': 21.9184091091156, 'throughput': 4.562374919738642, 'expert_loads_saved': 8256}, {'latency': 21.91471266746521, 'throughput': 4.563144473642173, 'expert_loads_saved': 9287}, {'latency': 21.94744563102722, 'throughput': 4.

## Plotting Performance Gain

In [ ]:
import pandas as pd
import numpy as np
import pickle

profiling_results = pickle.load(open("profiling_results.pkl", "rb"))

In [ ]:
records = []

for config_num in range(len(run_configs)):
    cfg = run_configs[config_num]
    
    result = profiling_results[config_num]
    
    latencies = []
    loads_saved = []
    throughputs = []
    
    for r in result:
        latencies.append(r['latency'])
        throughputs.append(r['throughput'])
        loads_saved.append(r['expert_loads_saved'])
    
    cfg['config_num'] = config_num
    cfg['mean_latency'] = np.mean(latencies)
    cfg['std_latency'] = np.std(latencies)
    cfg['mean_loads_saved'] = np.mean(loads_saved)
    cfg['std_loads_saved'] = np.std(loads_saved)
    cfg['mean_throughput'] = np.mean(throughputs)
    cfg['std_throughput'] = np.std(throughputs)
    
    records.append(cfg)

In [ ]:
df = pd.DataFrame.from_records(records)
df['label'] = df.apply(lambda x: f"{x['routing_strategy']}__{x['alpha']}" if 'THRESHOLDING' == x['routing_strategy'] else x['routing_strategy'], axis = 1)
df

In [ ]:
sub_dfs = {}
for offloads in [3, 4, 5, 6]:
    sub_dfs[offloads] = df[df['offload_per_layer'] == offloads]

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns

# plt.style.use('seaborn-darkgrid')
    
# Create a figure and an axis object
#fig, ax = plt.subplots()

# Get unique labels for color coding
labels = df['label'].unique()

# Assign colors to each label (can also use a colormap if preferred)
colors = plt.cm.get_cmap('tab10', len(labels))

# Iterate over each label to plot
for i, label in enumerate(labels):
    # Filter data for the current label
    sub_df = df[df['label'] == label]

    # Plot error bars
    ax1.errorbar(sub_df['offload_per_layer'], sub_df['mean_latency'],
                yerr=sub_df['std_latency'], label=label,
                color=colors(i), elinewidth=3, capsize=3)

# Labeling the axes
ax1.set_xlabel('Offloads per Layer')
ax1.set_ylabel('Latency')

ax1.set_xticks([3, 4, 5, 6])

# Add a title
ax1.set_title('Latency per sequence trends')

# Add legend
ax1.legend(title='Label', loc='upper left')

# Show plot
# plt.show()

# plt.style.use('seaborn-darkgrid')
    
# Create a figure and an axis object
# fig, ax = plt.subplots()

# Get unique labels for color coding
labels = df['label'].unique()

# Assign colors to each label (can also use a colormap if preferred)
colors = plt.cm.get_cmap('tab10', len(labels))

# Iterate over each label to plot
for i, label in enumerate(labels):
    # Filter data for the current label
    sub_df = df[df['label'] == label]

    # Plot error bars
    ax2.errorbar(sub_df['offload_per_layer'], sub_df['mean_throughput'],
                yerr=sub_df['std_throughput'], label=label,
                color=colors(i), elinewidth=3, capsize=3)

# Labeling the axes
ax2.set_xlabel('Offloads per Layer')
ax2.set_ylabel('Throughput')

ax2.set_xticks([3, 4, 5, 6])

# Add a title
ax2.set_title('Generation seconds/token trends')

# Add legend
# ax2.legend(title='Label')

# Show plot
# plt.show()

# plt.style.use('seaborn-darkgrid')
    
# Create a figure and an axis object
# fig, ax = plt.subplots()

# Get unique labels for color coding
labels = df['label'].unique()

# Assign colors to each label (can also use a colormap if preferred)
colors = plt.cm.get_cmap('tab10', len(labels))

# Iterate over each label to plot
for i, label in enumerate(labels):
    # Filter data for the current label
    sub_df = df[df['label'] == label]

    # Plot error bars
    ax3.errorbar(sub_df['offload_per_layer'], sub_df['mean_loads_saved'],
                yerr=sub_df['std_loads_saved'], label=label,
                color=colors(i), elinewidth=0, capsize=0)

# Labeling the axes
ax3.set_xlabel('Offloads per Layer')
ax3.set_ylabel('Expert loads saved')

ax3.set_xticks([3, 4, 5, 6])

# Add a title
ax3.set_title('Avg expert loads saved ')

# Add legend
# ax3.legend(title='Label')

# Show plot
plt.show()